# Install helm

Run the commands below to install helm, for installing postgresql, betydb, and PEcAn. 

In [ ]:
%%bash
mkdir -p ~/.local/bin/ ~/.local/src
curl -fsSL -o ~/.local/bin/get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
chmod 700 ~/.local/bin/get_helm.sh
env HELM_INSTALL_DIR=~/.local/bin ~/.local/bin/get_helm.sh --no-sudo
echo DONE

# Install PostgreSQL with helm

Run the commands below to install PostgreSQL relational database with helm. 

In [ ]:
%%bash
helm uninstall postgresql
# --version 11.9.13
helm install postgresql oci://registry-1.docker.io/bitnamicharts/postgresql \
  --set enabled=true \
  --set image.tag=14.5.0 \
  --set volumePermissions.securityContext.runAsUser="auto" \
  --set fullnameOverride=pecan-postgresql \
  --set auth.postgresqlPassword=supersecret \
  --set auth.postgresPassword=supersecret
#  --set securityContext.enabled=false \
#  --set containerSecurityContext.enabled=false \
#  --set shmVolume.chmod.enabled=false \
#  --set fullnameOverride=pecan-postgresql \
#  --set auth.username=postgres \
#  --set serviceAccount.enabled=true \
echo DONE

In [ ]:
%%bash
oc wait pod/pecan-postgresql-0 --for=condition=Ready --timeout=2m
echo DONE

# Install BetyDB with helm

Run the commands below to install BetyDB on top of PostgreSQL for ecological forecasting data. 

In [ ]:
%%bash
oc delete secret/pecan-betydb
oc create secret generic pecan-betydb \
  --from-literal=postgresqlPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=betyPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=secretKey="thisisnotasecret"
echo DONE

In [ ]:
%%bash
git clone https://github.com/computate-org/bety-helm.git ~/bety-helm/
(cd ~/bety-helm/ && helm dependency build)
oc delete job/pecan-betydb-load-db
helm uninstall betydb
helm install betydb ~/bety-helm/  \
  --set image.repository=quay.io/computateorg/bety \
  --set image.tag=latest \
  --set image.pullPolicy=Always \
  --set postgres.enabled=false \
  --set postgresql.enabled=false \
  --set postgresql.fullnameOverride=pecan-postgresql \
  --set postgresql.serviceAccount.enabled=true \
  --set postgresql.postgresqlHost=pecan-postgresql \
  --set serviceAccount.enabled=true \
  --set serviceAccount.name=pecan-betydb \
  --set externalSecret=true \
  --set nameOverride=pecan \
  --set fullnameOverride=pecan-betydb
echo DONE

In [ ]:
%%bash
oc wait job/pecan-betydb-load-db --for=condition=complete --timeout=10m
echo DONE

# Install PEcAn with helm

Run the commands below to install PEcAn for ecological forecasting. 

In [ ]:
%%bash
git clone https://github.com/computate-org/pecan-helm.git -b upgrade-pecan ~/pecan-helm/
helm repo add ncsa https://opensource.ncsa.illinois.edu/charts/
(cd ~/pecan-helm/ && helm dependency build)
helm uninstall pecan
helm install pecan ~/pecan-helm/ \
  --set docs.replicaCount=0 \
  --set executor.replicaCount=0 \
  --set models.biocro-0-95.replicaCount=0 \
  --set models.ed2-2-2-0.replicaCount=0 \
  --set models.maespa-git.replicaCount=0 \
  --set models.sipnet-git.replicaCount=0 \
  --set models.sipnet-136.replicaCount=0 \
  --set betydb.enabled=false \
  --set betydb.postgresql.enabled=false \
  --set betydb.postgresql.postgresqlHost=pecan-postgresql \
  --set betydb.postgresql.postgresqlUser=postgres \
  --set betydb.postgresql.postgresqlUsername=postgres \
  --set betydb.postgresql.postgresqlDatabase=postgres \
  --set betydb.postgresql.service.port=5432 \
  --set betydb.service.port=8000 \
  --set rabbitmq.persistence.size=3Gi \
  --set clustername=pecan-server \
  --set clusterfqdn=pecan.$(oc get route python -o jsonpath={.spec.host} | cut -d "." -f2-) \
  --set initializeData=true \
  --set ingress.enabled=true \
  --set pecan.env.betydb=http://betydb:8000/ \
  --set persistence.size=3Gi \
  --set betydb.betyUser=bety \
  --set betydb.betyDatabase=bety \
  --set betydb.ingress.hosts={pecan-betydb} \
  --set ingress.hosts={pecan.$(oc get route python -o jsonpath={.spec.host} | cut -d "." -f2-)}
echo DONE